In [1]:
#Version 1.6
import pygame
from pygame.locals import *
import random, math, sys
#start up pygame module
pygame.init()

#game board paramaters (pixel lengths)
height=580
width=980
margin=90
mu = 0.005

#set screen and refresh rate
Screen = pygame.display.set_mode((width,height),pygame.RESIZABLE)
clock = pygame.time.Clock()

#load external sprites ie. table and balls stored in assets folder
table=pygame.image.load('assets/Pool-Table-Above.png').convert()
ball_sprites=[]

#simply loads all ball images automatically do not mess with the quotation marks!
for i in range(1,16):
    loadimage='''=pygame.image.load('assets/ball_'''
    end='''.png') '''
    code=  'ball_'+str(i)+loadimage+str(i)+end
    exec(code)
    ball_sprites.append('ball_'+str(i))

#load cueball image separately to be used in cueball class
cueballimage=pygame.image.load('assets/ball_16.png')
    
#creates class for all ball objects except the cueball
class Ball:
    def __init__(self,x,y,speedx,speedy):  
        self.radius = radius
        self.x      = x
        self.y      = y
        self.speedx = speedx
        self.speedy = speedy

#triangle

#l is xdistance from bumper to the bottom left ball in pixels
l=200

#radius of balls(can be changed but will not change dimensions of sprites)
radius=12

#initial velocity of balls in triangle
vx=0
vy=0

#manually setting coordinates for each ball in triange sourced from Jatin Kumar Mandav's pool game
Ball1  = Ball(l           , height/2 - 4*radius, vx, vy)
Ball2  = Ball(l + 2*radius, height/2 - 3*radius, vx, vy)
Ball3  = Ball(l           , height/2 - 2*radius, vx, vy)
Ball4  = Ball(l + 4*radius, height/2 - 2*radius, vx, vy)
Ball5  = Ball(l + 2*radius, height/2 - 1*radius, vx, vy)
Ball6  = Ball(l           , height/2           , vx, vy)
Ball7  = Ball(l + 6*radius, height/2 - 1*radius, vx, vy)
Ball8  = Ball(l + 4*radius, height/2           , vx, vy)
Ball9  = Ball(l + 8*radius, height/2           , vx, vy)
Ball10 = Ball(l + 6*radius, height/2 + 1*radius, vx, vy)
Ball11 = Ball(l + 2*radius, height/2 + 1*radius, vx, vy)
Ball12 = Ball(l           , height/2 + 2*radius, vx, vy)
Ball13 = Ball(l + 4*radius, height/2 + 2*radius, vx, vy)
Ball14 = Ball(l + 2*radius, height/2 + 3*radius, vx, vy)
Ball15 = Ball(l           , height/2 + 4*radius, vx, vy)

#appends all ball objects (except cueball) to Balls list for use later
Balls=[]
for i in range(1,16):
    Balls.append(eval('Ball'+str(i)))


#Cueball
Cueball=Ball(width*2/3,height*.5,0,0)

#Cuestick

maxcueballspeed=.05
def Cuestick():

    mousex, mousey = pygame.mouse.get_pos()
    start=(Cueball.x, height-Cueball.y)
    end=(mousex,mousey)

    length=math.dist(start,end)

    force = length/30.0
    if force > 6:
        force = 6
    CueYDiff = height-Cueball.y - mousey
    CueXDiff = Cueball.x - mousex
    
    Angle= 90+(math.degrees(math.atan2((CueYDiff), (CueXDiff))))

    Cueball.speedx=force*math.sin(math.radians(Angle))
    Cueball.speedy=force*math.cos(math.radians(Angle))
  
    
#heavily modified from circlecollisions2.py to accomodate realistic 2d collisions
#function that deals with physics of collisions
def BallCollide(C1,C2):
    XDiff=C2.x-C1.x
    YDiff=C2.y-C1.y
    
#to avoid divide by zero error
    if C1.speedx==0:
        theta1=.5*math.pi
    else:
        theta1=math.atan(C1.speedy/C1.speedx)

    if C2.speedx==0:
        theta2=.5*math.pi
    else:
        theta2=math.atan(C2.speedy/C2.speedx)
        
    if XDiff==0:
        phi=0.5*math.pi
    else: 
        phi=math.atan(YDiff/XDiff)

    v1=math.sqrt((C1.speedx**2)+(C1.speedy**2))
    v2=math.sqrt((C2.speedx**2)+(C2.speedy**2))
    
#equations sourced from wikipedia elastic collisions page: https://en.wikipedia.org/wiki/Elastic_collision
#absolute value added so u are the components of the magnitude
    u1x = abs(v2*math.cos(theta2-phi)*math.cos(phi) + v1*math.sin(theta1-phi)*math.cos(phi+.5*math.pi))
    u1y = abs(v2*math.cos(theta2-phi)*math.sin(phi) + v1*math.sin(theta1-phi)*math.sin(phi+.5*math.pi))

    u2x = abs(v1*math.cos(theta1-phi)*math.cos(phi) + v2*math.sin(theta2-phi)*math.cos(phi+.5*math.pi))
    u2y = abs(v1*math.cos(theta1-phi)*math.sin(phi) + v2*math.sin(theta2-phi)*math.sin(phi+.5*math.pi))

#adjustments due to the nature of the arctan function using impact paramaters XDiff and YDiff
    if   XDiff > 0:

        if   YDiff > 0: #C2 above/right
            u1x= -u1x
            u1y= -u1y
            u2x= -u2x
            u2y=  u2y

        elif YDiff < 0: #C2 below/right 
            u1x= -u1x
            u1y=  u1y
            u2x= -u2x
            u2y= -u2y
        elif YDiff ==0:  #C2 head on from right
            u1x= -u1x
            u1y= u1y
            u2x= u2x
            u2y= u2y
    elif XDiff < 0:     
        if   YDiff > 0: #C2 above/left
            u1x=  u1x
            u1y= -u1y
            u2x=  u2x
            u2y=  u2y
        elif YDiff < 0: #C2 below/left
            u1x=  u1x
            u1y=  u1y
            u2x=  u2x
            u2y= -u2y
        elif YDiff ==0: #C2 head on from left
            u1x= u1x
            u1y= u1y
            u2x= u2x
            u2y= u2y
    elif XDiff == 0:
        if   YDiff > 0: #C2 head on from above
            u1x= u1x
            u1y= -u1y
            u2x= u2x
            u2y= u2y
        elif YDiff < 0: #c2 head on from below
            u1x= u1x
            u1y= u1y
            u2x= u2x
            u2y= u2y

    C1.speedx=u1x
    C1.speedy=u1y
    C2.speedx=u2x
    C2.speedy=u2y

#moves the balls each frame by continually adding the speed components
threshold=.1
def Move():
#friction
#Balls
    for Ball in Balls:
        if Ball.speedx==0:
            Angle=.5*math.pi
        else:
            Angle=math.atan(Ball.speedy/Ball.speedx)
        frictionx=abs(mu*math.cos(Angle))
        frictiony=abs(mu*math.sin(Angle))
        
        if abs(Ball.speedx)>0:
            temp1=abs(Ball.speedx)-frictionx
            if   Ball.speedx<0:
                 Ball.speedx= -temp1
            elif Ball.speedx>0:
                 Ball.speedx=  temp1
            elif temp1<0 or abs(Ball.speedx)<threshold:
                 Ball.speedx=0 
        else:
            Ball.speedx=0
            
        if abs(Ball.speedy)>0:
            temp2=abs(Ball.speedy)-frictiony
            if   Ball.speedy<0:
                 Ball.speedy= -temp2
            elif Ball.speedy>0:
                 Ball.speedy=  temp2
            elif temp2<0 or abs(Ball.speedy)<threshold:
                 Ball.speedy=0
        else:
            Ball.speedy=0

        Ball.x += round(Ball.speedx,2)
        Ball.y += round(Ball.speedy,2)
        
#CueBall
    if Cueball.speedx==0:
        Angle=.5*math.pi
    else:
        Angle = math.atan(Cueball.speedy/Cueball.speedx)
        
    frictionx = abs(mu*math.cos(Angle))
    frictiony = abs(mu*math.sin(Angle))
    
    if abs(Cueball.speedx)>0:
        temp3=abs(Cueball.speedx)-frictionx
        if   Cueball.speedx<0:
             Cueball.speedx= -temp3
        elif Cueball.speedx>0:
             Cueball.speedx=  temp3
        elif temp3<0 or abs(Cueball.speedx)<threshold:
             Cueball.speedx=0
    else:
        Cueball.speedx=0
        
    if abs(Cueball.speedy)>0:
        temp4=abs(Cueball.speedy)-frictiony
        if   Cueball.speedy<0:
             Cueball.speedy= -temp4
        elif Cueball.speedy>0:
             Cueball.speedy=  temp4
        elif temp4<0 or abs(Cueball.speedy)<threshold:
             Cueball.speedy=0
    else:
        Cueball.speedy=0
        
    Cueball.x += round(Cueball.speedx,2)
    Cueball.y += round(Cueball.speedy,2)
    
#Detects if Balls come in contact with another object heavily modified from circlecollisions2.py
def CollisionDetect():
    
    #defines centers of holes in list
    corner_holes_x=[80,80,900,900]
    corner_holes_y=[80,500,80,500]
    side_holes_x=[490,490]
    side_holes_y=[75,505]
    
    #balls(not including cueball)*
    
    #side holes
    for Ball in Balls:
        for j in range(2):       
            if math.sqrt(  ((Ball.x-side_holes_x[j])**2)  +  ((Ball.y-side_holes_y[j])**2)  ) <= (Ball.radius+25):
                PotCollide(Ball)
    #corner holes
    for Ball in Balls:
        for k in range(4):       
            if math.sqrt(  ((Ball.x-corner_holes_x[k])**2)  +  ((Ball.y-corner_holes_y[k])**2)  ) <= (Ball.radius+35):
                PotCollide(Ball)
    
    #walls
    for Ball in Balls: 
        if Ball.x <= Ball.radius+90 or Ball.x >= width-margin-Ball.radius:
            Ball.speedx *= -1
        if Ball.y <= Ball.radius+90 or Ball.y >= height-margin-Ball.radius:    
            Ball.speedy *= -1
    
    #collisions between balls
    Ball1_list=[]
    Ball2_list=[]
    
    #detects if balls are within 2*r of each other using distance formula
    counter=-1
    for Ball1 in Balls:
        for Ball2 in Balls:
            if Ball1 != Ball2:
                if math.sqrt(  ((Ball1.x-Ball2.x)**2)  +  ((Ball1.y-Ball2.y)**2)  ) <= (Ball1.radius+Ball2.radius):
    
    #detects if a pair of balls has already been ran through function. 
    #eliminates a pair: ball2 and ball1 that was already run through the function as a pair ball1 and ball2
                    counter+=1
                    Ball1_list.append(Ball1)
                    Ball2_list.append(Ball2)
                    Ball_list=[(Ball1_list[n], Ball2_list[n]) for n in range(0,len(Ball1_list))]
                    swapped = [(sub[1], sub[0]) for sub in Ball_list] 
                    if Ball_list[counter] in swapped:
                        continue
                    else:

                        Ball1Speed = math.sqrt((Ball1.speedx**2)+(Ball1.speedy**2))
                        Ball2Speed = math.sqrt((Ball2.speedx**2)+(Ball2.speedy**2))
                        if Ball1Speed<Ball2Speed:
                            BallCollide(Ball1,Ball2)
                        else:
                            BallCollide(Ball2,Ball1)
                            
    #cueball*                      
    
    #walls                       
    if Cueball.x <= Cueball.radius+margin or Cueball.x >= width-margin-Cueball.radius:
        Cueball.speedx *= -1
    if Cueball.y <= Cueball.radius+margin or Cueball.y >= height-margin-Cueball.radius:    
        Cueball.speedy *= -1
    
    #balls
    for Ball3 in Balls:
        if math.sqrt(  ((Cueball.x-Ball3.x)**2)  +  ((Cueball.y-Ball3.y)**2)  ) <= (Cueball.radius+Ball3.radius):
            CueballSpeed = math.sqrt((Cueball.speedx**2)+(Cueball.speedy**2))
            Ball3Speed   = math.sqrt((Ball3.speedx**2)  +(Ball3.speedy**2))
            if CueballSpeed<Ball3Speed:
                BallCollide(Cueball,Ball3)
            elif CueballSpeed>Ball3Speed:
                BallCollide(Ball3,Cueball)

#moves balls that have collided with pocket outside of game window (not including cueball)
def PotCollide(D1):
    D1.x=-20
    D1.y=-20
    D1.speedx=0
    D1.speedy=0

#actually shows all images on the screen
def Draw():
    index=0
    Screen.blit(table,(0,0))

    #Balls
    for Ball in Balls:
        Screen.blit(eval(ball_sprites[index]),(int(Ball.x-Ball.radius),int(580-Ball.y-Ball.radius)))
        index+=1
    
    #CueStick
    if abs(Cueball.speedx)<=maxcueballspeed and abs(Cueball.speedy)<=maxcueballspeed:
        mousex, mousey = pygame.mouse.get_pos()
        pygame.draw.line(Screen, (0, 0, 0), (mousex, mousey), (Cueball.x, height-Cueball.y), 5)
         
    #CueBall
    Screen.blit(cueballimage,(int(Cueball.x-Cueball.radius),int(580-Cueball.y-Cueball.radius)))
    
    # flips coordinate system
    pygame.display.flip()

#detects user input
def GetInput():
    keystate = pygame.key.get_pressed()
    for event in pygame.event.get():
        if event.type == QUIT or keystate[K_ESCAPE]:
            pygame.quit(); sys.exit()
        if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    pygame.quit(); sys.exit()  
        if abs(Cueball.speedx)<=maxcueballspeed and abs(Cueball.speedy)<=maxcueballspeed:
            if event.type == pygame.MOUSEBUTTONDOWN:
                Cuestick()
    
#actually all runs of the code continuously       
while True:
    GetInput()
    CollisionDetect()
    Move()
    Draw()

# Sourced some code from:
#ball bouncing simulator, https://www.pygame.org/project/619/1056
#an actual pygame poolgame, https://jatinmandav.wordpress.com

#Sourced Physics from: 
#wikipedia, https://en.wikipedia.org/wiki/Elastic_collision

pygame 2.0.1 (SDL 2.0.14, Python 3.8.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/Users/antoniomolteni/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
